### Spark Moive Recommendation
In this notebook, we will use an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies in [MovieLens small dataset](https://grouplens.org/datasets/movielens/latest/)

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [3]:
os.environ["PYSPARK_PYTHON"] = "python3"

## Part0: Data Import

In [5]:
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
movies_df = spark.read.load("/FileStore/tables/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("/FileStore/tables/ratings.csv", format='csv', header = True)
links_df = spark.read.load("/FileStore/tables/links.csv", format='csv', header = True)
tags_df = spark.read.load("/FileStore/tables/tags.csv", format='csv', header = True)

## Part I: Exploratory Data Analysis

In [8]:
tmp1 = ratings_df.groupBy('userId').count().select(F.min('count')).collect()[0][0]
print('Minimum number of ratings per user is {}'.format(tmp1))

Minimum number of ratings per user is 20

In [9]:
tmp2 = ratings_df.groupBy('movieId').count().select(F.min('count')).collect()[0][0]
print('Minimum number of ratings per movie is {}'.format(tmp2))

Minimum number of ratings per movie is 1

In [10]:
tmp1 = ratings_df.groupBy("movieId").count()
tmp1 = tmp1.filter(tmp1['count']==1).count()
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

3446 out of 9724 movies are rated by only one user

In [11]:
tmp1 = ratings_df.groupBy("movieId").count().select(F.avg('count')).collect()[0][0]
print('Average number of ratings per movie is {}'.format(tmp1))

Average number of ratings per movie is 10.369806663924312

In [12]:
tmp2 = ratings_df.groupBy("userId").count().select(F.avg('count')).collect()[0][0]
print('Average number of ratings per user is {}'.format(tmp2))

Average number of ratings per user is 165.30491803278687

In [13]:
movie_num = movies_df.select('movieId').distinct().count()
user_num = ratings_df.select('userId').distinct().count()
rating_num = ratings_df.select('rating').count()
print('Total number of users is {}\n'.format(user_num))
print('Total number of movies is {}\n'.format(movie_num))
print('Total number of ratings is {}'.format(rating_num))

Total number of users is 610

Total number of movies is 9742

Total number of ratings is 100836

In [15]:
# The sparsity of the movie ratings
# sparcity = 1- rating_num/(movie_num*user_num)
denominator = movie_num*user_num
numerator = rating_num
sparsity = (1-numerator/denominator)*100
print ("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is 98.30% empty.

In [16]:
movie_num_rated = ratings_df.select('movieId').distinct().count()
print('Total number of movies rated is {}'.format(movie_num_rated))

Total number of movies rated is 9724

In [17]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

### Movies not rated before.

In [19]:
spark.sql(
  '''
SELECT m.*,r.rating
FROM movies m 
    LEFT JOIN ratings r ON m.movieId = r.movieId
WHERE r.rating is Null
'''
).show(5)

+-------+--------------------+--------------------+------+
movieId| title| genres|rating|
+-------+--------------------+--------------------+------+
 1076|Innocents, The (1...|Drama|Horror|Thri...| null|
 2939| Niagara (1953)| Drama|Thriller| null|
 3338|For All Mankind (...| Documentary| null|
 3456|Color of Paradise...| Drama| null|
 4194|I Know Where I'm ...| Drama|Romance|War| null|
+-------+--------------------+--------------------+------+
only showing top 5 rows

### Movie Genres

Each movie belongs to more than 1 genre,as shown below. We need to seperate these genres.

In [22]:
spark.sql(
  '''
SELECT DISTINCT title, genres
FROM movies
  '''
).show(5)

+--------------------+--------------------+
 title| genres|
+--------------------+--------------------+
Color of Night (1...| Drama|Thriller|
Surviving the Gam...|Action|Adventure|...|
Little Big League...| Comedy|Drama|
 Whatever (1998)| Drama|
 Ronin (1998)|Action|Crime|Thri...|
+--------------------+--------------------+
only showing top 5 rows

In [23]:
## Data processing to seperate the genres for a movie
genres_pd_df = spark.sql("SELECT DISTINCT title, genres FROM movies").toPandas()
genres_pd_df['genres'] = genres_pd_df['genres'].apply(lambda x:x.split('|'))
genres_pd_df = pd.concat([genres_pd_df['title'],genres_pd_df['genres'].apply(pd.Series)],axis = 1).set_index('title')
genres_sep_pd_df = genres_pd_df.stack().reset_index(level=0)
genres_sep_pd_df.columns = ['title','genre']

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

We store all the movie genres in a list called movie_genres_list.

In [25]:
movie_genres_list = genres_sep_pd_df['genre'].unique().tolist()
print('Here are all the movie genres:'+'\n', movie_genres_list)

Here are all the movie genres:
 ['Drama', 'Thriller', 'Action', 'Adventure', 'Comedy', 'Crime', 'Film-Noir', 'Horror', 'Mystery', 'Romance', 'War', 'Sci-Fi', 'IMAX', 'Animation', 'Fantasy', 'Documentary', 'Children', 'Musical', 'Western', '(no genres listed)']

### Movie for Each Category

In [27]:
genres_sep_df = sqlContext.createDataFrame(genres_sep_pd_df)
genres_sep_df.registerTempTable('genres_sep')

In [28]:
spark.sql(
  '''
SELECT genre,count(*) AS count
FROM genres_sep
GROUP BY 1
ORDER BY 2 DESC
'''
).show(5)

+--------+-----+
 genre|count|
+--------+-----+
 Drama| 4361|
 Comedy| 3756|
Thriller| 1894|
 Action| 1828|
 Romance| 1596|
+--------+-----+
only showing top 5 rows

We put each movie under its genre. This is stored in a dictionary called movie_genre_dict, where the key is the genre, and the values is a list that contains the movies belonging to this genre.

In [30]:
movie_genre_dict = dict()
for _ in movie_genres_list:
  movie_genre_dict[_] = list()
for index, row in genres_sep_pd_df.iterrows():
   movie_genre_dict[row["genre"]].append(row["title"])

Let's take a look at the Film-Noir genre as an example.

In [32]:
print('Here are movies that belong to the crime genre:'+'\n',movie_genre_dict['Film-Noir'])

Here are movies that belong to the crime genre:
 ['Angel Heart (1987)', 'D.O.A. (1988)', "Killer's Kiss (1955)", 'Undercurrent (1946)', 'Killers, The (1946)', 'Strange Love of Martha Ivers, The (1946)', 'Night of the Hunter, The (1955)', 'Lost Highway (1997)', '13 Tzameti (2005)', 'Fury (1936)', 'Renaissance (2006)', 'Out of the Past (1947)', 'This Gun for Hire (1942)', 'Dark Passage (1947)', 'Grifters, The (1990)', 'Strangers on a Train (1951)', 'Hoodlum (1997)', 'High Sierra (1941)', 'Drive (2011)', 'Laura (1944)', 'Wrong Man, The (1956)', 'Patlabor: The Movie (Kidô keisatsu patorebâ: The Movie) (1989)', 'Key Largo (1948)', 'Crossfire (1947)', 'Devil in a Blue Dress (1995)', 'While the City Sleeps (1956)', 'Sorry, Wrong Number (1948)', 'Long Goodbye, The (1973)', 'Brick (2005)', "Miller's Crossing (1990)", 'Postman Always Rings Twice, The (1946)', 'Rififi (Du rififi chez les hommes) (1955)', 'Bullet to the Head (2012)', 'Maltese Falcon, The (1941)', 'Pickup on South Street (1953)', 'L.A. Confidential (1997)', 'Foreign Correspondent (1940)', 'Johnny Eager (1942)', 'Mildred Pierce (1945)', 'White Heat (1949)', 'Gilda (1946)', 'Call Northside 777 (1948)', 'Big Sleep, The (1946)', 'Third Man, The (1949)', 'Notorious (1946)', 'Dark City (1998)', 'Sin City (2005)', 'Suture (1993)', 'House of Games (1987)', 'Letter, The (1940)', 'Bitter Moon (1992)', 'Trou, Le (Hole, The) (Night Watch, The) (1960)', 'Tinker Tailor Soldier Spy (2011)', 'Stranger, The (1946)', 'You Only Live Once (1937)', 'Angels with Dirty Faces (1938)', 'M (1931)', 'Down by Law (1986)', 'Lady from Shanghai, The (1947)', 'Limits of Control, The (2009)', 'Suddenly (1954)', 'Mulholland Drive (2001)', 'Born to Kill (1947)', 'Chinatown (1974)', 'Killing, The (1956)', 'Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)', 'Too Late for Tears (1949)', '2 Days in the Valley (1996)', 'Sweet Smell of Success (1957)', 'Suspicion (1941)', 'In a Lonely Place (1950)', 'Double Life, A (1947)', 'Touch of Evil (1958)', 'Blood Simple (1984)', 'Pure Formality, A (Pura formalità, Una) (1994)', 'Asphalt Jungle, The (1950)', 'High and Low (Tengoku to jigoku) (1963)', 'D.O.A. (1950)', 'Harder They Fall, The (1956)', 'Detour (1945)', 'Kiss Me Deadly (1955)', 'Rubber (2010)', 'One False Move (1992)', 'I Am a Fugitive from a Chain Gang (1932)', 'This World, Then the Fireworks (1997)', 'Double Indemnity (1944)', 'Stray Dog (Nora inu) (1949)']

## Part2: Spark ALS based approach for training model
We will use an Spark ML to predict the ratings, so let's reload "ratings.csv" using ``sc.textFile`` and then convert it to the form of (user, item, rating) tuples.

In [34]:
ratings_df.show(5)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

In [35]:
movie_ratings_df=ratings_df.drop('timestamp')
movie_ratings_df.show(5)

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 1| 1| 4.0|
 1| 3| 4.0|
 1| 6| 4.0|
 1| 47| 5.0|
 1| 50| 5.0|
+------+-------+------+
only showing top 5 rows

In [36]:
# Data type convert
from pyspark.sql.types import IntegerType, FloatType
movie_ratings_df = movie_ratings_df.withColumn("userId", movie_ratings_df["userId"].cast(IntegerType()))
movie_ratings_df = movie_ratings_df.withColumn("movieId", movie_ratings_df["movieId"].cast(IntegerType()))
movie_ratings_df = movie_ratings_df.withColumn("rating", movie_ratings_df["rating"].cast(FloatType()))

In [37]:
movie_ratings_df.show(5)

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 1| 1| 4.0|
 1| 3| 4.0|
 1| 6| 4.0|
 1| 47| 5.0|
 1| 50| 5.0|
+------+-------+------+
only showing top 5 rows

### ALS Model Selection and Evaluation

With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [39]:
# import package
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

In [40]:
#Create test and train set
(training,test)=movie_ratings_df.randomSplit([0.8,0.2],seed = 42)

## We will tune the hyperparameters using ParamGridBuilder and CrossValidator.

In [42]:
#Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",coldStartStrategy = 'drop',nonnegative = True, implicitPrefs = False)
# Confirm that a model called "als" was created
type(als)

Out[26]: pyspark.ml.recommendation.ALS

In [43]:
#Tune model using ParamGridBuilder
# We will just tune rank and regParam considering long run time, after we get the have combination, we will use larger iterations.
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [3,5,10]) \
            .addGrid(als.maxIter, [10]) \
            .addGrid(als.regParam, [0.05,0.15,0.25]) \
            .build()
print ("Num models to be tested: ", len(param_grid))

Num models to be tested: 9

In [44]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 

In [45]:
# Build Cross validation 
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

In [46]:
#Fit ALS model to training data
model = cv.fit(training)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [47]:
#Extract best model from the tuning exercise using ParamGridBuilder
best_model = model.bestModel

In [48]:
#Generate predictions and evaluate using RMSE
predictions=best_model.transform(test)
rmse = evaluator.evaluate(predictions)

In [49]:
#Print evaluation metrics and model parameters
print ("RMSE = "+str(rmse))
print ("**Best Model**")
print (" Rank:",best_model._java_obj.parent().getRank())   #parent()method will return an estimator,you can get the best params then
print (" MaxIter:",best_model._java_obj.parent().getMaxIter())
print (" RegParam:",best_model._java_obj.parent().getRegParam()) 

RMSE = 0.8736197761693623
**Best Model**
 Rank: 10
 MaxIter: 10
 RegParam: 0.15

### Increase iteration number and Model testing

In [51]:
#Increase the interation for the best ALS model
#coldStartStrategy = 'drop' is important, otherwise, you will recieve rmse = nan
#Spark allows users to set the coldStartStrategy parameter to “drop” in order to drop any rows in the DataFrame of predictions that contain NaN values. 
als_50 = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",rank = 10, maxIter = 50,regParam = 0.15, nonnegative = True, coldStartStrategy = 'drop',implicitPrefs = False)
#fit the model to training data
best_model_50 = als_50.fit(training)

In [52]:
#generate predictions on test data
prediction_50 = best_model_50.transform(test)

In [53]:
#tell spark how to evaluate predictions
evaluator_50 = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
#obtain rmse
rmse_50 = evaluator_50.evaluate(prediction_50)
#print rmse
print('RMSE=',rmse_50)

RMSE= 0.8705602687772467

In [54]:
prediction_50_pd_df = prediction_50.toPandas()[['movieId','rating','prediction']].set_index('movieId')
display(prediction_50_pd_df.plot(style=['o','rx']))

In [55]:
# round the prediction to a scale of 1-5
def round_to_5scale(x):
  if x<=round(x) and 0.5<=x<=5:
    return round(x)-0.5 if round(x)-x>0.25 else round(x)
  if x>round(x) and 0.5<=x<=5:
    return round(x)+0.5 if x-round(x)>0.25 else round(x)
  if x>5:
    return 5
  if x<0.5:
    return 0.5

In [56]:
prediction_50_pd_df['prediction'] = prediction_50_pd_df['prediction'].apply(lambda x: round_to_5scale(x))
display(prediction_50_pd_df.plot(style=['o','rx']))

In [57]:
prediction_50_user_pd_df = prediction_50.toPandas()[['userId','rating','prediction']].set_index('userId')
prediction_50_user_pd_df['prediction'] = prediction_50_user_pd_df['prediction'].apply(lambda x: round_to_5scale(x))
display(prediction_50_user_pd_df.plot(style=['o','rx']))

### Apply model to all data and see the performance

In [59]:
#Best_model RMSE
alldata=best_model.transform(movie_ratings_df)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 0.6975433153657589

In [60]:
#Best_model_50 RMSE
alldata=best_model_50.transform(movie_ratings_df)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 0.6874073412892818

In [61]:
alldata.registerTempTable("alldata")

In [62]:
sparl.sql(
  '''
SELECT *
FROM movies
	JOIN alldata ON movies.movieId = alldata.movieId
  '''
).show(5)

movieId,title,genres,userId,movieId,rating,prediction
148,"Awfully Big Adventure, An (1995)",Drama,191,148,5.0,4.855351
471,"Hudsucker Proxy, The (1994)",Comedy,133,471,4.0,2.416814
471,"Hudsucker Proxy, The (1994)",Comedy,597,471,2.0,3.3838356
471,"Hudsucker Proxy, The (1994)",Comedy,385,471,4.0,2.9433992
471,"Hudsucker Proxy, The (1994)",Comedy,436,471,3.0,3.3313468
471,"Hudsucker Proxy, The (1994)",Comedy,602,471,4.0,3.1600466
471,"Hudsucker Proxy, The (1994)",Comedy,91,471,1.0,2.5822668
471,"Hudsucker Proxy, The (1994)",Comedy,409,471,3.0,3.0681155
471,"Hudsucker Proxy, The (1994)",Comedy,372,471,3.0,2.7291358
471,"Hudsucker Proxy, The (1994)",Comedy,599,471,2.5,2.4970746


In [63]:
alldata_pd_df = alldata.toPandas()[['movieId','rating','prediction']].set_index('movieId')
alldata_pd_df.plot(style=['o','rx'])
display()

In [64]:
alldata_pd_df['prediction'] = alldata_pd_df['prediction'].apply(lambda x: round_to_5scale(x))
alldata_pd_df.plot(style=['o','rx'])
display()

In [65]:
alldata_user_pd_df = alldata.toPandas()[['userId','rating','prediction']].set_index('userId')
alldata_user_pd_df['prediction'] = alldata_user_pd_df['prediction'].apply(lambda x: round_to_5scale(x))
display(alldata_user_pd_df.plot(style=['o','rx']))

### Let's look more closely on the user and movie data to decide which user to recommend.

In [67]:
ratings_info_df = movie_ratings_df.groupBy('movieId').avg('rating')
movie_ratings_count= movie_ratings_df.groupBy('movieId').count()
ratings_info_df = ratings_info_df.join(movie_ratings_count,'movieId','left').join(movies_df,'movieId','left')
ratings_info_df.show(3)

+-------+-----------------+-----+--------------------+--------------------+
movieId| avg(rating)|count| title| genres|
+-------+-----------------+-----+--------------------+--------------------+
 1580|3.487878787878788| 165|Men in Black (a.k...|Action|Comedy|Sci-Fi|
 2366| 3.64| 25| King Kong (1933)|Action|Adventure|...|
 3175| 3.58| 75| Galaxy Quest (1999)|Adventure|Comedy|...|
+-------+-----------------+-----+--------------------+--------------------+
only showing top 3 rows

### UserId order by rating count

In [69]:
ratings_df.groupBy("userId").count().toPandas().sort_values(by = 'count',ascending=False).head()

,userId,count
345,414,2698
472,599,2478
280,474,2108
28,448,1864
193,274,1346


We will recommend user 414 and 599 as they have the highest rating count. This means the recommendations to them would be more reliable.

### Histogram of rating counts

In [72]:
display(ratings_info_df.toPandas()['count'].hist(bins=50,log= True))

### Histogram of the ratings

In [74]:
display(ratings_info_df.toPandas()['avg(rating)'].hist(bins=50))

In [75]:
fig, ax = plt.subplots()
g = sns.jointplot(x='avg(rating)', y='count', data=ratings_info_df.toPandas())
display(g.fig)

## Recommend moive to users with id: 414, 599.

In [77]:
# use the recommendation function of ALS
ALS_recommendations = best_model.recommendForAllUsers(10)
ALS_recommendations.filter(ALS_recommendations['userId'] == 599).show()

+------+--------------------+
userId| recommendations|
+------+--------------------+
 599|[[96004, 4.252195...|
+------+--------------------+

We need to process the above dataframe for readability.

In [79]:
# Data procesing of the ALS_recommendations dataframe
from pyspark.sql.functions import explode,col
recommendations_df = (ALS_recommendations\
                      .select("userId",\
                              explode("recommendations")\
                              .alias("recommendation"))\
                      .select("userId", "recommendation.movieId",\
                              col("recommendation.rating")\
                              .alias('prediction')))
recommendations_df.show(3)

+------+-------+----------+
userId|movieId|prediction|
+------+-------+----------+
 471| 6818| 5.1408505|
 471| 8477| 5.0905204|
 471| 3379| 4.981934|
+------+-------+----------+
only showing top 3 rows

In [80]:
# we only recommend movies that have not been watched by users before
recommendations_df = recommendations_df.join(movies_df,["movieId"],"left").join(ratings_df,['movieId','UserId'],'left')
recommendations_df = recommendations_df.drop('timestamp')
recommendations_df = recommendations_df.filter(ratings_df.rating.isNull())
recommendations_df.show()

+-------+------+----------+--------------------+--------------------+------+
movieId|userId|prediction| title| genres|rating|
+-------+------+----------+--------------------+--------------------+------+
 6818| 471| 5.1408505|Come and See (Idi...| Drama|War| null|
 8477| 471| 5.0905204| Jetée, La (1962)| Romance|Sci-Fi| null|
 3379| 471| 4.981934| On the Beach (1959)| Drama| null|
 96004| 471| 4.981934|Dragon Ball Z: Th...|Action|Adventure|...| null|
 5490| 471| 4.858012| The Big Bus (1976)| Action|Comedy| null|
 5915| 471| 4.858012|Victory (a.k.a. E...| Action|Drama|War| null|
 132333| 471| 4.858012| Seve (2014)| Documentary|Drama| null|
 40491| 471| 4.846524|Match Factory Gir...| Comedy|Drama| null|
 26326| 471| 4.6848273|Holy Mountain, Th...| Drama| null|
 102217| 471| 4.677115|Bill Hicks: Revel...| Comedy| null|
 96004| 463| 5.278501|Dragon Ball Z: Th...|Action|Adventure|...| null|
 3379| 463| 5.278501| On the Beach (1959)| Drama| null|
 60943| 463| 5.0922623| Frozen River (2008)| Drama| null|
 3567| 463| 4.9940257| Bossa Nova (2000)|Comedy|Drama|Romance| null|
 33649| 463| 4.925018| Saving Face (2004)|Comedy|Drama|Romance| null|
 92494| 463| 4.8854833|Dylan Moran: Mons...| Comedy|Documentary| null|
 102217| 463| 4.8854833|Bill Hicks: Revel...| Comedy| null|
 33779| 463| 4.8854833|Eddie Izzard: Dre...| Comedy| null|
 171495| 463| 4.8617916| Cosmos| (no genres listed)| null|
 93988| 463| 4.814151|North & South (2004)| Drama|Romance| null|
+-------+------+----------+--------------------+--------------------+------+
only showing top 20 rows

We will only recommend movies that the users haven't watched/rated.

movies recommendations for user 414

In [82]:
recommendations_df.filter(recommendations_df['userId'] == 414).show()

+-------+------+----------+--------------------+--------------------+------+
movieId|userId|prediction| title| genres|rating|
+-------+------+----------+--------------------+--------------------+------+
 96004| 414| 5.301862|Dragon Ball Z: Th...|Action|Adventure|...| null|
 3379| 414| 5.301862| On the Beach (1959)| Drama| null|
 6818| 414| 5.1921563|Come and See (Idi...| Drama|War| null|
 8477| 414| 5.1837234| Jetée, La (1962)| Romance|Sci-Fi| null|
 132333| 414| 5.026645| Seve (2014)| Documentary|Drama| null|
 5915| 414| 5.026645|Victory (a.k.a. E...| Action|Drama|War| null|
 5490| 414| 5.026645| The Big Bus (1976)| Action|Comedy| null|
 4495| 414| 4.941838|Crossing Delancey...| Comedy|Romance| null|
 92494| 414| 4.9203644|Dylan Moran: Mons...| Comedy|Documentary| null|
 102217| 414| 4.9203644|Bill Hicks: Revel...| Comedy| null|
+-------+------+----------+--------------------+--------------------+------+

movies recommendations for user 599

In [84]:
recommendations_df.filter(recommendations_df['userId'] == 599).show()

+-------+------+----------+--------------------+------------------+------+
movieId|userId|prediction| title| genres|rating|
+-------+------+----------+--------------------+------------------+------+
 5490| 599| 3.9496071| The Big Bus (1976)| Action|Comedy| null|
 5915| 599| 3.9496071|Victory (a.k.a. E...| Action|Drama|War| null|
 132333| 599| 3.9496071| Seve (2014)| Documentary|Drama| null|
 60943| 599| 3.9102483| Frozen River (2008)| Drama| null|
 4495| 599| 3.8764865|Crossing Delancey...| Comedy|Romance| null|
 6818| 599| 3.8465486|Come and See (Idi...| Drama|War| null|
 92494| 599| 3.8462806|Dylan Moran: Mons...|Comedy|Documentary| null|
+-------+------+----------+--------------------+------------------+------+

In [85]:
# Another way is to recommendForUserSubset function
#users = ALS_recommendations.filter(ALS_recommendations['userId'] == 575)
#ALS_recommendations_target = best_model.recommendForUserSubset(users,1)
#ALS_recommendations_target.show()

## Find the similar moives for moive with id: 464, 471

The similarites of different movies can be recognized from correlations of their user ratings. Let's say user A and B both gave movie 1,2 5 star rating. This indicates that the movie 1 and 2 might be highily similar.

However, we have a challenge in that some of the movies have very few ratings and may end up having high correlation simply because one or two people gave them a 5 star rating. We can fix this by setting a threshold for the number of ratings. From the histogram earlier we saw a sharp decline in number of ratings from 100. Therefore we will choose this as our threshold.

In [88]:
movie_matrix = movie_ratings_df.toPandas().pivot_table(index='userId', columns='movieId', values='rating')

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [89]:
def find_similar_movie(x):
  movie_x_rating = movie_matrix[x]
  similar_to_x=movie_matrix.corrwith(movie_x_rating).reset_index(level=0)
  similar_to_x.dropna(axis = 0,how = 'any',inplace=True)
  similar_to_x.columns = ['movieId','correlation']
  
  similar_to_x_df = sqlContext.createDataFrame(similar_to_x)
  similar_to_x_movie = similar_to_x_df.join(ratings_info_df,'movieId','left').toPandas()[['movieId','correlation','title','count']]
  res = similar_to_x_movie[similar_to_x_movie['count']>100].sort_values(by = 'correlation',ascending = False)
  return similar_to_x,res

We will only find similar movies that have been rated more than 100 times.

movies similar to movie 471

In [91]:
# Movies similar to 471
corr_471,similar_to_471_movie = find_similar_movie(471)

/databricks/python/lib/python3.7/site-packages/numpy/lib/function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
 c = cov(x, y, rowvar)
/databricks/python/lib/python3.7/site-packages/numpy/lib/function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
 c *= np.true_divide(1, fact)
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [92]:
corr_471.plot(kind='scatter',x = 'movieId',y='correlation')
display()

In [93]:
similar_to_471_movie.head()

,movieId,correlation,title,count
2426,79132,0.973124,Inception (2010),143
1788,48516,0.724861,"Departed, The (2006)",107
2576,33794,0.677430,Batman Begins (2005),116
1350,1222,0.608101,Full Metal Jacket (1987),102
1229,364,0.570549,"Lion King, The (1994)",172


movies similar to movie 464

In [95]:
# Movies similar to 464
corr_464,similar_to_464_movie = find_similar_movie(464)

/databricks/python/lib/python3.7/site-packages/numpy/lib/function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
 c = cov(x, y, rowvar)
/databricks/python/lib/python3.7/site-packages/numpy/lib/function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
 c *= np.true_divide(1, fact)
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [96]:
corr_464.plot(kind='scatter',x = 'movieId',y='correlation')
display()

In [97]:
similar_to_464_movie.head()

,movieId,correlation,title,count
362,5816,1.000000,Harry Potter and the Chamber of Secrets (2002),102
528,4896,0.904534,Harry Potter and the Sorcerer's Stone (a.k.a. ...,107
1465,58559,0.866025,"Dark Knight, The (2008)",149
450,253,0.853913,Interview with the Vampire: The Vampire Chroni...,109
389,7153,0.845154,"Lord of the Rings: The Return of the King, The...",185


## Report
In this project, the movie data from movie lens data set which includes about 600 users and 9500 movies were analyzed to gain insights on movie recomendations to users and finding similar movies. At the beginning, we calculate the sparisity of the movie ratings which is 98.3%, this tells us that the ratings dataframe is mostly empty, which brings significance to predict the user ratings from what we have.

To achieve the goal,the data was analyzed on Spark platform from perfoming data cleaning,processing to model training with Alternating Least Squares (ALS) algorithm.During which, grid search and cross validation were applied to tune the hyperparameters. It is found that using large rank and iterzations would help achieve a low RMSE. Finally, we choose a rank of 10, iterate 50 times and regPram = 0.15,a RMSE of 0.69 was achieved.This means that on average the model predicts 0.69 above or below values of the original ratings matrix.

By successsfully predicting the ratings using the best model, we not only fill the rating dataframe and recommend our users with movies they have never watched, but also find similar movies through their correlations. This brings huge business value to the company.